In [15]:
# Import required packages
import random
import pandas as pd
from faker import Faker
import datetime
from datetime import date

In [16]:

# Define number of locations of each type
num_ports = 1
num_customer_dcs = 2
num_customer_stores = 30
num_carrier_warehouses = 1
# Define number of trailers of each type
num_trailers = 75
num_refrigs = 5
num_city_delivs = 75
# Define number of trailers of each type
num_drivers = 150
pct_class1_driver=0.7


Set orders , equipments, drivers and locations

In [17]:
import warnings
warnings.filterwarnings('ignore')

# Initialize Faker
fake = Faker()

# Set seed for reproducibility
random.seed(42)

# Function to generate random latitude and longitude around a given point
def generate_coords(lat, long, delta=0.1):
    return round(lat + random.uniform(-delta, delta), 4), round(long + random.uniform(-delta, delta), 4)

# Create Locations DataFrame
locations_data = {
    'Location Name': [],
    'Location Type': ['Port']*num_ports + ['Customer DC']*num_customer_dcs + ['Customer Store']*num_customer_stores + ['Carrier Warehouse']*num_carrier_warehouses,
    'Location Latitude': [],
    'Location Longitude': []
}

for i, loc_type in enumerate(locations_data['Location Type']):
    loc_name = f'{loc_type} Location {i+1}'
    locations_data['Location Name'].append(loc_name)
    lat, long = generate_coords(45.5019, -73.5674)
    locations_data['Location Latitude'].append(lat)
    locations_data['Location Longitude'].append(long)

locations_df = pd.DataFrame(locations_data)

# Create Trailers DataFrame
trailers_data = {
    'Equipment ID': [f'TR-{str(i).zfill(3)}' for i in range(1, num_trailers + 1)] + [f'RE-{str(i).zfill(3)}' for i in range(1, num_refrigs + 1)] + [f'CD-{str(i).zfill(3)}' for i in range(1, num_city_delivs + 1)],
    'Equipment Type': ['Trailer']*num_trailers + ['Refrigerated Truck']*num_refrigs + ['City Delivery']*num_city_delivs,
    'Capacity': [2000]*num_trailers + [2000]*num_refrigs + [700]*num_city_delivs
}

trailers_df = pd.DataFrame(trailers_data)

# Create Orders DataFrame
orders_data = {
    'Order ID': [],
    'Pickup Date': [],
    'Lead Time': [],
    'Volume': [],
    'Event': []   
}

# Generate dates and orders for the past 2 years, with 75-120 orders each day
start_date = datetime.datetime.today() - datetime.timedelta(days=2 * 365)
dates = pd.date_range(start=start_date, end=datetime.datetime.today() + datetime.timedelta(days=14))

# Define trend and seasonality patterns
trend_pattern = [85, 90, 95, 100, 105, 110, 115, 120]
seasonality_pattern = [0.8, 0.9, 1.1, 1.2, 1.3, 1.4, 1.3, 1.2, 1.1, 1.0, 0.9, 0.8]

for i, date in enumerate(dates):
    # Calculate trend value based on the pattern
    trend_value = trend_pattern[i % len(trend_pattern)]

    # Calculate seasonality value based on the pattern
    
    seasonality_value = seasonality_pattern[date.month - 1]

    # Calculate the number of orders based on trend and seasonality
    num_orders = int((trend_value * seasonality_value))

    for _ in range(num_orders):
        orders_data['Order ID'].append(len(orders_data['Order ID']) + 1)
        orders_data['Pickup Date'].append(date)
        orders_data['Lead Time'].append(random.randint(0, 5))
        orders_data['Volume'].append(random.randint(200, 2000))
        orders_data['Event'].append(0)

orders_df = pd.DataFrame(orders_data)

# Add other fields based on initial fields
orders_df['Delivery Type'] = orders_df['Lead Time'].apply(lambda x: 'Not Direct' if x > 1 else 'Direct')
orders_df['Delivery Date'] = (pd.to_datetime(orders_df['Pickup Date']) + pd.to_timedelta(orders_df['Lead Time'], unit='d')).dt.date
orders_df['Pickup Address'] = random.choices(locations_df[locations_df['Location Type'].isin(['Port', 'Customer DC'])]['Location Name'].values, k=len(orders_df))
orders_df['Delivery Location'] = random.choices(locations_df[locations_df['Location Type'] == 'Customer Store']['Location Name'].values, k=len(orders_df))

# Set all orders to 'Not Delivered'
orders_df['Status'] = 'Not Delivered'

# Add Trailer Requirement based on Volume
def trailer_requirement(volume):
    if volume < 700:
        return 'City Delivery'
    elif random.random() < 0.05:   # 5% probability of needing a refrigerated trailer
        return 'Refrigerated Truck'
    else:
        return 'Trailer'

orders_df['Trailer Requirement'] = orders_df['Volume'].apply(trailer_requirement)


# Driver information
drivers_data = {
    'Driver ID': [f'DR-{str(i).zfill(3)}' for i in range(1, num_drivers + 1)],
    'Driver Name': [fake.name() for _ in range(num_drivers)],
    'Hired Date': [fake.date_between(start_date='-5y', end_date='today') for _ in range(num_drivers)],
    'License Type': ['Class 1' if random.random() < pct_class1_driver else 'Class 2' for _ in range(num_drivers)],
}

drivers_df = pd.DataFrame(drivers_data)

In [18]:
# Add seniority column to drivers_df
today = datetime.date.today()
drivers_df['Seniority'] = (today - pd.to_datetime(drivers_df['Hired Date']).dt.date).dt.days // 365

# Function to assign vacation days based on seniority
def assign_vacation_days(seniority):
    if seniority < 1:
        return 2
    elif seniority < 2:
        return 3
    elif seniority < 3:
        return 4
    elif seniority < 4:
        return 5
    else:
        return 6

# Assign vacation days
drivers_df['Vacation Days'] = drivers_df['Seniority'].apply(assign_vacation_days)

# Initialize vacations DataFrame
vacations_df = pd.DataFrame(columns=['Driver ID', 'Start Date', 'End Date'])

# Generate random vacation dates
for _, driver in drivers_df.iterrows():
    for _ in range(driver['Vacation Days']):
        start_date = fake.date_between(start_date='-1y', end_date='today')
        end_date = start_date + datetime.timedelta(days=random.randint(1, 7))  # vacation lasts 1-7 days
        vacations_df = vacations_df.append({
            'Driver ID': driver['Driver ID'],
            'Start Date': start_date,
            'End Date': end_date
        }, ignore_index=True)


In [19]:
# Initialize Unavailability DataFrame
unavailability_data = {
    'Date': [],
    'Type': [],  # 'Equipment'
    'ID': []  # Equipment ID
}
unavailability_df = pd.DataFrame(unavailability_data)

# Each day, randomly select 5% of equipment to be unavailable
dates = pd.date_range(start=orders_df['Pickup Date'].min(), end=orders_df['Pickup Date'].max())

for date in dates:
    # Equipment
    num_equipment_unavailable = round(0.05 * len(trailers_df))  # 5% of total equipment
    unavailable_equipment = trailers_df.sample(num_equipment_unavailable)['Equipment ID'].tolist()
    
    for equipment_id in unavailable_equipment:
        unavailability_df = unavailability_df.append({
            'Date': date,
            'Type': 'Equipment',
            'ID': equipment_id
        }, ignore_index=True)

unavailability_df.head()


,Date,Type,ID
0,2021-07-24 16:33:11.057231,Equipment,CD-001
1,2021-07-24 16:33:11.057231,Equipment,TR-040
2,2021-07-24 16:33:11.057231,Equipment,TR-009
3,2021-07-24 16:33:11.057231,Equipment,CD-039
4,2021-07-24 16:33:11.057231,Equipment,RE-004


In [44]:

# Initialize Schedule DataFrame
schedule_data = {
    'Date': [],
    'Driver ID': [],
    'Equipment ID': [],
    'Equipment Available': [],
    'License Type': [],
    'Equipment Type': []
}

schedule_df = pd.DataFrame(schedule_data)
# Sort drivers DataFrame so that 'Class 1' drivers come first
sorted_drivers_df = drivers_df.sort_values(by='License Type', ascending=False)

# Determine the earliest and latest dates from the orders
start_date = orders_df['Pickup Date'].min()
end_date = orders_df['Delivery Date'].max()

# Assign equipment to drivers based on their license type
for _, driver in sorted_drivers_df.iterrows():
    # Assign equipment to each driver for every day in the order date range
    date_range = pd.date_range(start=start_date, end=end_date)

    for date in date_range:
        # Check if this driver has already been assigned equipment for this day
        if not schedule_df[(schedule_df['Date'] == date) & (schedule_df['Driver ID'] == driver['Driver ID'])].empty:
            continue

        # Choose a random available equipment that the driver is qualified to operate
        if driver['License Type'] == 'Class 1':  
            available_equipment = trailers_df[
                ~trailers_df['Equipment ID'].isin(schedule_df[schedule_df['Date'] == date]['Equipment ID'])
            ]
        else:  # 'Class 2'
            available_equipment = trailers_df[
                (trailers_df['Equipment Type'] == 'City Delivery') &
                ~trailers_df['Equipment ID'].isin(schedule_df[schedule_df['Date'] == date]['Equipment ID'])
            ]

        # If there is no available equipment, note 'No Equipment' for this driver
        if available_equipment.empty:
            assigned_equipment_id = None
            assigned_equipment_type = None
            equipment_available = False
        else:
            assigned_equipment = available_equipment.sample(1).iloc[0]
            assigned_equipment_id = assigned_equipment['Equipment ID']
            assigned_equipment_type = assigned_equipment['Equipment Type']
            equipment_available = True

        # Add the assignment to the schedule
        schedule_df = schedule_df.append({
            'Date': date,
            'Driver ID': driver['Driver ID'],
            'Equipment ID': assigned_equipment_id,
            'Equipment Available': equipment_available,
            'License Type': driver['License Type'],
            'Equipment Type': assigned_equipment_type
        }, ignore_index=True)


In [46]:
# Initialize new columns for the Driver ID, Equipment ID and Actual Delivery Date in the order DataFrame
orders_df['Driver ID'] = None
orders_df['Equipment ID'] = None
orders_df['Actual Delivery Date'] = None

# Assign drivers and equipment to orders based on equipment requirement, availability, and capacity
for _, order in orders_df.iterrows():
    # Only assign drivers to orders that have a pickup date in the past
    if order['Pickup Date'].date() > datetime.datetime.today().date():
        continue

    # Get available assignments (driver-equipment pairs) for this order
    available_assignments = schedule_df[
        (schedule_df['Date'] == order['Pickup Date']) &
        (schedule_df['Equipment Type'] == order['Trailer Requirement']) &
        (schedule_df['Equipment Available'] == 1)
    ]

    # If there are no available assignments, reschedule this order to the next day
    if available_assignments.empty:
        rescheduled_date = order['Pickup Date'] + datetime.timedelta(days=1)
        orders_df.at[_, 'Pickup Date'] = rescheduled_date
        orders_df.at[_, 'Actual Delivery Date'] = rescheduled_date  # Update the actual delivery date
        orders_df.at[_, 'Event'] = 3
        orders_df.at[_, 'Not Delivered Reason'] = 'No Available Driver-Equipment Pair'
        continue

    # Choose a random assignment
    assigned_assignment = available_assignments.sample(1).iloc[0]

    # Update the status of the chosen assignment in the schedule to 'Delivered'
    schedule_df.at[assigned_assignment.name, 'Status'] = 'Delivered'

    # Add the assignment to the orders DataFrame
    orders_df.at[_, 'Driver ID'] = assigned_assignment['Driver ID']
    orders_df.at[_, 'Equipment ID'] = assigned_assignment['Equipment ID']
    orders_df.at[_, 'Actual Delivery Date'] = order['Pickup Date']  # Actual delivery date is the same as the pickup date
    orders_df.at[_, 'Status'] = 'Delivered'


In [56]:
# Initialize new columns for Cost and Revenue in the order DataFrame
orders_df['Cost'] = 0.0
orders_df['Revenue'] = 0.0

# Generate events and calculate cost and revenue
for i, order in orders_df.iterrows():
    # Get driver experience
    driver_experience_values = drivers_df.loc[drivers_df['Driver ID'] == order['Driver ID'], 'Seniority'].values

    if driver_experience_values.size > 0:
        driver_experience = driver_experience_values[0]
    else:
        driver_experience = 0  # or some other default value

    # Basic cost and revenue for delivered orders
    if order['Status'] == 'Delivered':
        orders_df.at[i, 'Cost'] = 5 * order['Volume'] + 200  # cost per unit of volume + overhead cost
        orders_df.at[i, 'Revenue'] = 10 * order['Volume']  # revenue per unit of volume

    # Generate event based on correlations and adjust cost and revenue for non-rescheduled orders
    if order['Event'] != 3:
        if order['Volume'] > 1900 and random.random() < 0.05:  # 5% chance of cancellation for very high volume orders
            orders_df.at[i, 'Event'] = 4
            orders_df.at[i, 'Cost'] += 5000  # arbitrary penalty for cancellation
        elif order['Trailer Requirement'] == 'Refrigerated Truck' and random.random() < 0.02:  # 2% chance of damage for refrigerated truck orders
            orders_df.at[i, 'Event'] = 5
            orders_df.at[i, 'Cost'] += 2000  # arbitrary penalty for damage
        elif driver_experience < 2 and random.random() < 0.01:  # 1% chance of accident for inexperienced drivers
            orders_df.at[i, 'Event'] = 6
            orders_df.at[i, 'Cost'] += 5000  # arbitrary penalty for accident
        elif order['Volume'] > 1000 and random.random() < 0.1:  # 10% chance of complaint for high volume orders
            orders_df.at[i, 'Event'] = 2
            orders_df.at[i, 'Cost'] += 500  # arbitrary penalty for complaint
        elif order['Lead Time'] > 3 and random.random() < 0.2:  # 20% chance of delay for long lead time orders
            orders_df.at[i, 'Event'] = 1
            orders_df.at[i, 'Cost'] += 100  # arbitrary penalty for delay

events_data = {
    'Event ID': [0, 1, 2, 3, 4, 5,6],
    'Event Description': ['No issues', 'Delay', 'Complaint','Rescheduled', 'Cancellation', 'Damages', 'Accidents']
}
event_description_df = pd.DataFrame(events_data)


In [58]:
# Save to Excel
with pd.ExcelWriter('EfficientRide_Dataset.xlsx', engine='openpyxl') as writer:
    locations_df.to_excel(writer, sheet_name='Locations', index=False)
    trailers_df.to_excel(writer, sheet_name='Trailers Pool', index=False)
    orders_df.to_excel(writer, sheet_name='Orders', index=False)
    drivers_df.to_excel(writer, sheet_name='Drivers', index=False)
    schedule_df.to_excel(writer, sheet_name='Schedule', index=False)
    vacations_df.to_excel(writer, sheet_name='Vacations', index=False)
    event_description_df.to_excel(writer, sheet_name='Event Description', index=False)
    unavailability_df.to_excel(writer, sheet_name='unavailability', index=False)